# Tworzymy nazwane klastry

W lekcji tej użyjemy LLM-ów do pomocy przy znalezieniu odpowiednich nazw i opisów dla klastrów.

In [9]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [10]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [11]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()
df.describe()

,age,edu_level,fav_animals,fav_place,gender
count,229,229,229,200,227
unique,8,3,5,4,2
top,35-44,Wyższe,Psy,Nad wodą,Mężczyzna
freq,83,184,123,73,171


ładujemy model wytrenowany w poprzedniej lekcji

In [12]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [13]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster
Cluster 3    38
Cluster 1    34
Cluster 0    30
Cluster 6    30
Cluster 5    29
Cluster 4    26
Cluster 2    23
Cluster 7    19
Name: count, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

poniższego kodu (kod1) nie rozumiem - zapytać Kuby

In [14]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [15]:
print(cluster_descriptions["Cluster 5"])

age - 45-54: 12, 35-44: 6, 55-64: 5, 25-34: 3, 18-24: 1, >=65: 1, unknown: 1, <18: 0
edu_level - Wyższe: 29, Podstawowe: 0, Średnie: 0
fav_animals - Koty: 10, Inne: 8, Brak ulubionych: 7, Koty i Psy: 4, Psy: 0
fav_place - Nad wodą: 15, Inne: 1, W górach: 0, W lesie: 0
gender - Mężczyzna: 25, Kobieta: 3



poniższego kodu (kod2) nie rozumiem - zapytać Kuby

In [16]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 7:
age - 35-44: 8, 45-54: 4, 25-34: 3, 18-24: 2, <18: 1, >=65: 1, 55-64: 0, unknown: 0
edu_level - Średnie: 18, Podstawowe: 1, Wyższe: 0
fav_animals - Psy: 13, Koty: 3, Brak ulubionych: 2, Inne: 1, Koty i Psy: 0
fav_place - Nad wodą: 12, W lesie: 2, Inne: 0, W górach: 0
gender - Mężczyzna: 13, Kobieta: 6


Klaster Cluster 3:
age - 45-54: 18, 35-44: 8, 25-34: 7, 55-64: 4, >=65: 1, 18-24: 0, <18: 0, unknown: 0
edu_level - Wyższe: 38, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 23, Brak ulubionych: 5, Inne: 5, Koty: 5, Koty i Psy: 0
fav_place - W lesie: 38, Inne: 0, Nad wodą: 0, W górach: 0
gender - Mężczyzna: 29, Kobieta: 9


Klaster Cluster 4:
age - 45-54: 9, 25-34: 7, 18-24: 6, 35-44: 4, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Średnie: 26, Podstawowe: 0, Wyższe: 0
fav_animals - Koty: 9, Inne: 7, Psy: 7, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, W lesie: 5, Inne: 3, Nad wodą: 0
gender - Mężczyzna: 24, Kobieta

In [17]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

To poniżej robi łądniejszego jsona bo coś

In [18]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [19]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [20]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Miłośnicy Wody z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Górscy Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują góry jako miejsce wypoczynku. Dominują osoby w wieku 45-54 lat, a większość z nich to mężczyźni. Ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Kociarze z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują góry i mają szczególne upodobanie do kotów. Większość z nich to mężczyźni.'}, 'Cluster 3': {'name': 'Leśni Entuzjaści z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują lasy jako miejsce wypoczynku. Dominują osoby